**ENSEMBLE LEARNING.**

Technique createsmultiple models and then combines them to produce better results than any of the single models

**Random Forest**
        - Can be used for classification and regression
        - Easily handles outlier
        - Accepts various types of inputs(Continuos, ordinal)
        - Less likely to overfit
        - Outputs feature importances.

In [53]:
import string 
import pandas as pd
import nltk
from nltk.tokenize import wordpunct_tokenize,word_tokenize
import os
import re

dataset = pd.read_csv(os.getcwd()+'\\Ex_Files_NLP_Python_ML_EssT\\Exercise Files\\Ch01\\01_03\\Start\\SMSSpamCollection.tsv', header=None, sep='\t')
dataset.columns = ['label', 'body_text']
dataset.sample(5)

#create new features containing the percentage of punctuation in text and the body length of text 

def count_punctuation_and_body_len(text):
    punct_count = [char for char in text if char in string.punctuation]
    body_len = len(text) - text.count(" ")
    punct_perc = round(len(punct_count)/body_len, 3)*100
    return body_len, punct_perc

sp_words = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
def clean_data(text):
    tokens = ''.join([char for char in text if char not in string.punctuation])
    tokens_punct = re.split('\s+', tokens)
    tokens_stop = [token.lower()  for token in tokens_punct if token not in sp_words]
    clean_text = [ps.stem(word) for word in tokens_stop]
    return clean_text

dataset['cleaned_text'] = dataset['body_text'].apply(lambda x: clean_data(x))
dataset['body_len'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[0])
dataset['punc%'] = dataset['body_text'].apply(lambda x: count_punctuation_and_body_len(x)[1])    
dataset.head()

,label,body_text,cleaned_text,body_len,punc%
0,ham,I've been searching for the right words to tha...,"[ive, search, right, word, thank, breather, i,...",160,2.5
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entri, 2, wkli, comp, win, fa, cup, fin...",128,4.7
2,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, dont, think, goe, usf, live, around, ...",49,4.1
3,ham,Even my brother is not like to speak with me. ...,"[even, brother, like, speak, they, treat, like...",62,3.2
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,"[i, have, a, date, on, sunday, with, will]",28,7.1


In [59]:
#vectorizing
from sklearn.feature_extraction.text import TfidfVectorizer

def return_cleaned_text_string(text):
    string_version = ''.join(text)
    return string_version

dataset['body_text_new'] = dataset['cleaned_text'].apply(lambda x: return_cleaned_text_string(x))

tfidf_vect = TfidfVectorizer(analyzer=clean_data)
x_tfidf = tfidf_vect.fit_transform(dataset['body_text'])
dataset_frame = pd.DataFrame(x_tfidf.toarray())
x_features = pd.concat([dataset[['body_len','punc%']], dataset_frame], axis=1)
x_features.head()

,body_len,punc%,0,1,2,3,4,5,6,7,...,8265,8266,8267,8268,8269,8270,8271,8272,8273,8274
0,160,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
from sklearn.ensemble import RandomForestClassifier

print (dir(RandomForestClassifier))
print (RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_make_estimator', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0,

In [ ]:
RUNNING RANDOM FOREST 

In [60]:
#randomforest in action
from sklearn.model_selection import KFold, cross_val_score

rf = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf, x_features, dataset['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([ 0.97217235,  0.97127469,  0.97127469,  0.96046721,  0.97124888])

In [35]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [61]:
X_train, X_test, y_train, y_test = train_test_split(x_features, dataset['label'], test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[:10]

[(0.069080512716422821, 'body_len'),
 (0.037360906493643473, 3146),
 (0.029531117807788821, 5784),
 (0.029422685895495411, 6051),
 (0.026788616642330548, 6350),
 (0.026719016342786076, 2025),
 (0.024638360085304845, 1337),
 (0.021714342157930552, 1791),
 (0.020475009758148172, 7433),
 (0.017680769713600501, 4842)]

In [42]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
print ('Precision: {} / Recall: {} / fscore: {} / Accuracy: {}'.format(round(precision, 3),
                                                                      round(recall, 3),
                                                                      round(fscore, 3),
                                                                      round((y_pred == y_test).sum()/len(y_pred), 3)))

Precision: 1.0 / Recall: 0.609 / fscore: 0.757 / Accuracy: 0.945


In [44]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth)
    rf_model = rf.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} --- Precision: {} / Recall: {} / Accuracy: {}'.format(n_est, depth,
                                                                              round(precision,3), 
                                                                              round(recall, 3),
                                                                            round((y_pred == y_test).sum()/len(y_pred), 3)))
    

In [45]:
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth: 10 --- Precision: 1.0 / Recall: 0.391 / Accuracy: 0.915
Est: 10 / Depth: 20 --- Precision: 1.0 / Recall: 0.609 / Accuracy: 0.945
Est: 10 / Depth: 30 --- Precision: 0.99 / Recall: 0.641 / Accuracy: 0.949
Est: 10 / Depth: None --- Precision: 0.992 / Recall: 0.795 / Accuracy: 0.97
Est: 50 / Depth: 10 --- Precision: 1.0 / Recall: 0.276 / Accuracy: 0.899
Est: 50 / Depth: 20 --- Precision: 1.0 / Recall: 0.571 / Accuracy: 0.94
Est: 50 / Depth: 30 --- Precision: 1.0 / Recall: 0.737 / Accuracy: 0.963
Est: 50 / Depth: None --- Precision: 1.0 / Recall: 0.827 / Accuracy: 0.976
Est: 100 / Depth: 10 --- Precision: 1.0 / Recall: 0.244 / Accuracy: 0.894
Est: 100 / Depth: 20 --- Precision: 1.0 / Recall: 0.603 / Accuracy: 0.944
Est: 100 / Depth: 30 --- Precision: 1.0 / Recall: 0.737 / Accuracy: 0.963
Est: 100 / Depth: None --- Precision: 1.0 / Recall: 0.827 / Accuracy: 0.976


In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [52]:
rf = RandomForestClassifier()
param = {'n_estomators': [10, 50, 300], 'max_depth':[30, 60, 90, None]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(x_features, dataset['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\Lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\user\Anaconda3\lib\runpy.py i

In [ ]:
rf = RandomForestClassifier()
param = {'n_estomators': [10, 50, 300], 'max_depth':[30, 60, 90, None]}
gs = GridSearchCV(rf, param, cv=5, n_jobs=-1)
gs_fit = gs.fit(x_features, dataset['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]